Readme

the ratio larger and smaller is base on the variable "step", so every time it might different. 

In line 124 'step += 0.001', if we add very small step number (like 0.00001 or smaller) for each iteration which it should have ratio very close 1. 

However, it might take longer time to compile. 

Also, the result of solution may be different every time because the list is randomly shuffled each time. 

use the print(results) to check

Therefore, we will different label and result every time

In [28]:
from collections import deque
import pickle
import random

def bfs(tree, start):
    visited = set([start])
    queue = deque([(start, 0)])
    distances = {}
    
    while queue:
        node, distance = queue.popleft()
        distances[node] = distance
        
        for neighbor in tree[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, distance + 1))
    
    return distances

def calculate_m_values(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        for h in range(len(tree)):
            if len([x for x in distances.values() if x <= h]) >= k:
                m_values.append(h)
                break
    
    return m_values

def greedy_labeling(tree, k, m_values):
    nodes = sorted(range(len(tree)), key=lambda x: m_values[x])
    labels = [None] * len(tree)
    
    step = 0
    while True:
        for node in nodes:
            if labels[node] is None:
                available_labels = set(range(k))
                distances = bfs(tree, node)
                neighbors = [x for x in distances if distances[x] <= m_values[node]]
                neighbors_p = neighbors[1:]
                random.shuffle(neighbors_p)
                neighbors[1:] = neighbors_p
                
                for neighbor in neighbors:
                    if labels[neighbor] in available_labels:
                        available_labels.remove(labels[neighbor])
                    if not available_labels:
                        break
                
                available_labels = list(available_labels)
                random.shuffle(available_labels)
                
                if available_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] is None:
                            labels[neighbor] = available_labels.pop()
                        if not available_labels:
                            break
                        
                unique_labels = set()
                if available_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] in unique_labels:
                            labels[neighbor] = available_labels.pop()
                            if not available_labels:
                                break
                        unique_labels.add(labels[neighbor])
            else:
                missing_labels = set(range(k))
                distances = bfs(tree, node)
                neighbors = [x for x in distances if distances[x] <= m_values[node]]
                neighbors_p = neighbors[1:]
                random.shuffle(neighbors_p)
                neighbors[1:] = neighbors_p
                
                for neighbor in neighbors:
                    if labels[neighbor] in missing_labels:
                        missing_labels.remove(labels[neighbor])
                    if not missing_labels:
                        break
                
                missing_labels = list(missing_labels)
                random.shuffle(missing_labels)
                
                if not missing_labels:
                    continue
                else:
                    for neighbor in neighbors:
                        if labels[neighbor] is None:
                            labels[neighbor] = missing_labels.pop()
                        if not missing_labels:
                            break
                        
                    unique_labels = set()
                    if missing_labels:
                        for neighbor in neighbors:
                            if labels[neighbor] in unique_labels:
                                labels[neighbor] = missing_labels.pop()
                                if not missing_labels:
                                    break
                            unique_labels.add(labels[neighbor])
            
        count = 0
        for node in nodes:
            missing_labels = set(range(k))
            distances = bfs(tree, node)
            neighbors = [x for x in distances if distances[x] <= m_values[node]+step]

            for neighbor in neighbors:
                if labels[neighbor] in missing_labels:
                    missing_labels.remove(labels[neighbor])
                if not missing_labels:
                    break

            if not missing_labels:
                count += 1
            
        if count == len(tree):
            break
        else:
            step += 0.001

    for i in range(len(labels)):
        if labels[i] is None:
            labels[i] = 0
            
    return labels

def load_input(file_adjlist, file_k_values):
    with open(file_adjlist, 'rb') as f:
        adj_list_of_trees = pickle.load(f)
    with open(file_k_values, 'rb') as f:
        k_values = pickle.load(f)
    
    return zip(adj_list_of_trees, k_values)    

def save_output(file_name, output_list):
    with open(file_name, 'wb') as f:
        pickle.dump(output_list, f)

def main():
    # Load input instances
    # data_input = load_input('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values')
    # data_input = load_input('Small_Examples_of_AdjLists_of_Trees', 'Small_Examples_of_k_values')
    # data_input = load_input('Medium_Examples_of_AdjLists_of_Trees', 'Medium_Examples_of_k_values')
    # data_input = load_input('Large_Examples_of_AdjLists_of_Trees', 'Large_Examples_of_k_values')

    # Load input test set
    data_input = load_input('Test_Set_Small_AdjLists_of_Trees', 'Test_Set_Small_of_k_values')
    # data_input = load_input('Test_Set_Medium_AdjLists_of_Trees', 'Test_Set_Medium_of_k_values')
    # data_input = load_input('Test_Set_Large_AdjLists_of_Trees', 'Test_Set_Large_of_k_values')

    # Run algorithm and save results
    
    results = []
    for tree, k in data_input:
        m_values = calculate_m_values(tree, k)
        labels = greedy_labeling(tree, k, m_values)
        results.append(labels)
    
    #check result
    print(results)

    # save_output('example_solutions', results)
    # save_output('small_solutions', results)
    # save_output('medium_solutions', results)
    # save_output('large_solutions', results)

if __name__ == '__main__':
    main()

[[7, 1, 3, 8, 6, 2, 5, 9, 4, 0, 7, 4, 0, 9, 3, 5, 0, 2, 6, 0, 3, 2, 0, 3, 7, 1, 0, 8, 9, 5, 5, 0, 1, 5, 8, 2, 1, 7, 7, 9, 6, 3, 0, 4, 2, 8, 6, 7, 8, 7, 2, 0, 6, 9, 4, 3, 2, 3, 1, 8, 4, 0, 1, 9, 0, 3, 6, 0, 8, 9, 1, 2, 3, 6, 1, 0, 0, 0, 0, 3, 8, 0, 0, 0, 1, 4, 0], [3, 2, 0, 2, 1, 0, 2, 2, 1, 0, 2, 2, 3, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 3, 3, 0, 1, 3, 1, 2, 1, 0, 3, 0, 2, 0], [0, 4, 4, 6, 1, 2, 7, 5, 3, 2, 5, 7, 1, 3, 2, 0, 0, 7, 3, 0, 1, 3, 0, 0, 0, 0, 0, 0, 2, 7, 1, 3, 4, 6, 0, 1, 5, 0, 2, 7, 6, 4, 6, 4, 6, 6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 4, 0, 5, 2, 7, 6, 1, 0, 1, 5, 6, 1, 0, 5, 6, 7, 0, 2, 3, 5, 7, 1, 6, 3, 2, 3, 5, 0, 7, 2, 4, 5, 7, 2, 4, 3, 5, 6, 4, 0, 2, 7, 0, 0, 0], [2, 4, 1, 6, 3, 5, 0, 0, 0, 2, 5, 3, 6, 3, 1, 5, 1, 6, 5, 0, 1, 6, 0, 2, 6, 4, 0, 2, 1, 4, 6, 5, 2, 4, 3, 6, 2, 3, 0, 2, 0, 0, 1, 4, 4, 0, 4, 0, 0, 0, 0, 0, 4, 1, 4, 0, 0, 0, 0, 6, 0, 0, 0, 0, 6, 6, 0, 0, 2, 6, 1, 5, 5, 1, 0, 0,

In [29]:
def bfs(tree, start):
    visited = set()
    queue = [(start, 0)]
    distances = {}

    while queue:
        current, depth = queue.pop(0)
        visited.add(current)
        distances[current] = depth
        for neighbor in tree[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth + 1))

    return distances

def r(tree, k, solution):
    r_values = []
    for node, label in enumerate(solution):
        label_set = set()
        distances = bfs(tree, node)
        h = 0
        while len(label_set) < k and h < len(tree):
            label_set.update({solution[i] for i, dist in distances.items() if dist <= h})
            h += 1
        r_values.append(h - 1)
    return r_values

def m(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        h = 0
        while len([1 for i, dist in distances.items() if dist <= h]) < k:
            h += 1
        m_values.append(h)
    return m_values

def compute_proximity_ratios(file_adjlist, file_k_values, file_solutions):
    input_instances = load_input(file_adjlist, file_k_values)
    solutions = load_solutions(file_solutions)

    proximity_ratios = []
    for (tree, k), solution in zip(input_instances, solutions):
        r_values = r(tree, k, solution)
        m_values = m(tree, k)

        ratios = [rv / mv for rv, mv in zip(r_values, m_values)]
        max_ratio = max(ratios)
        proximity_ratios.append(max_ratio)

    return proximity_ratios

def load_solutions(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def main():
    # Compute proximity ratios for small solutions
    # proximity_ratios = compute_proximity_ratios('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values', 'example_solutions')
    # proximity_ratios = compute_proximity_ratios('Small_Examples_of_AdjLists_of_Trees', 'Small_Examples_of_k_values', 'small_solutions')
    # proximity_ratios = compute_proximity_ratios('Medium_Examples_of_AdjLists_of_Trees', 'Medium_Examples_of_k_values', 'medium_solutions')
    # proximity_ratios = compute_proximity_ratios('Large_Examples_of_AdjLists_of_Trees', 'Large_Examples_of_k_values', 'large_solutions')

    #test set
    # proximity_ratios = compute_proximity_ratios('Test_Set_Small_AdjLists_of_Trees', 'Test_Set_Small_of_k_values', 'small_solutions')
    # proximity_ratios = compute_proximity_ratios('Test_Set_Medium_AdjLists_of_Trees', 'Test_Set_Medium_of_k_values', 'medium_solutions')
    proximity_ratios = compute_proximity_ratios('Test_Set_Large_AdjLists_of_Trees', 'Test_Set_Large_of_k_values', 'large_solutions')
    print("Proximity ratios:", proximity_ratios)
    print("Maximum ratio:", max(proximity_ratios))

if __name__ == '__main__':
    main()
    # sols = load_solutions('small_solutions')
    # for sol in sols:
    #     print(sol)

Proximity ratios: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Maximum ratio: 2.0
